In [2]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_974b9ee0b7264eda853e2d397d28058e = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='enK26baM-pxTCVXwkUF0yUs6pSJxwm4HMWTok5Yv-CVu',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_974b9ee0b7264eda853e2d397d28058e.get_object(Bucket='housing-donotdelete-pr-dhqo3ofsz9hwbs',Key='housing.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']

df_data_1 = pd.read_csv(body, header=None, delimiter=r"\s+", names=column_names)
df_data_1.head()


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [3]:
column_sets = ['LSTAT', 'PTRATIO', 'INDUS', 'TAX', 'AGE', 'ZN', 'RM']
X = df_data_1.loc[:,column_sets]
y = df_data_1['MEDV']


In [4]:
import sklearn.model_selection as model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.65,test_size=0.35, random_state=45)


In [5]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
pipeline_org = Pipeline(steps=[("classifier",LinearRegression())])
pipeline_org.fit(X_train, y_train)

Pipeline(steps=[('classifier', LinearRegression())])

In [6]:
import pickle
pickle.dump( pipeline_org, open( "housing-prediction-model.pkl", 'wb') )
!mkdir model-dir
!cp housing-prediction-model.pkl model-dir
!tar -zcvf housing-prediction-model.tar.gz housing-prediction-model.pkl

housing-prediction-model.pkl


In [7]:
from sklearn.metrics import mean_squared_error
import numpy as np
predictions = pipeline_org.predict( X_test )
lin_mse = mean_squared_error(y_test, predictions)
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

5.2670055904574475


In [8]:
api_key = 'KuHIixPi_pNOi6b9DkX9rewhOm-US-kxcmB9xHKxaT1k'
location = 'https://eu-de.ml.cloud.ibm.com'
wml_credentials = {
    "apikey": api_key,
    "url": location
}

In [9]:
!pip install -U ibm-watson-machine-learning

Requirement already up-to-date: ibm-watson-machine-learning in /opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages (1.0.53)


In [10]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

In [11]:
space_id = '11392c54-0523-44c1-a972-901faf315a87'
client.spaces.list(limit=10)
client.set.default_space(space_id)

------------------------------------  -------------  ------------------------
ID                                    NAME           CREATED
11392c54-0523-44c1-a972-901faf315a87  housing_space  2021-03-07T12:48:12.734Z
------------------------------------  -------------  ------------------------


'SUCCESS'

In [15]:
import json

sofware_spec_uid = client.software_specifications.get_id_by_name("default_py3.7")
metadata = {
    client.repository.ModelMetaNames.NAME: "housing-LinReg",
    client.repository.ModelMetaNames.TYPE: 'scikit-learn_0.23',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid
}
model_details_pkl = client.repository.store_model( model="model-dir", meta_props=metadata )


In [16]:
published_model_uid = client.repository.get_model_uid(model_details_pkl)
model_details = client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

{
  "entity": {
    "software_spec": {
      "id": "e4429883-c883-42b6-87a8-f419d64088cd",
      "name": "default_py3.7"
    },
    "type": "scikit-learn_0.23"
  },
  "metadata": {
    "created_at": "2021-03-07T13:11:37.620Z",
    "id": "c1d79352-42b6-4bdb-abfb-60741049984a",
    "modified_at": "2021-03-07T13:11:40.311Z",
    "name": "housing-LinReg",
    "owner": "IBMid-550005A6UW",
    "space_id": "11392c54-0523-44c1-a972-901faf315a87"
  },
  "system": {
    "warnings": []
  }
}


In [17]:

metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Deployment of housing model",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

created_deployment = client.deployments.create(published_model_uid, meta_props=metadata)



#######################################################################################

Synchronous deployment creation for uid: 'c1d79352-42b6-4bdb-abfb-60741049984a' started

#######################################################################################


initializing
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='cb42e408-84ee-43a0-a75f-1188e0d737b3'
------------------------------------------------------------------------------------------------




In [18]:
scoring_endpoint = client.deployments.get_scoring_href(created_deployment)
print(scoring_endpoint)

https://eu-de.ml.cloud.ibm.com/ml/v4/deployments/cb42e408-84ee-43a0-a75f-1188e0d737b3/predictions
